In [1]:
# Mount the Google Drive Storage to this notebook

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Importing necessary libraries and modules
import zipfile      # Used for extracting zip files
import os           # Provides functions to interact with the operating system (colab's)
import shutil       # Module used to delete files and directories
import json         # Provides functions for working with JSON data
import numpy as np  # NumPy, a library for working with arrays and numerical data
import cv2          # OpenCV, a library for computer vision tasks

In [3]:
# Provide the name of your zip file that was uploaded.
zip_file_name = '/content/drive/MyDrive/project-1-at-2024-09-05-15-24-8491b307.zip'  # Replace this with the actual file name

# Create a directory where the contents of the zip file will be extracted.
extract_to_dir = '/content'
os.makedirs(extract_to_dir, exist_ok=True)  # This will make sure the directory exists. If it doesn't, it creates it.

# Now, let's unzip the file.
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_to_dir)  # This extracts everything in the zip file to the specified directory.

# Print confirmation that the extraction was successful.
print(f'Files extracted to {extract_to_dir}')


Files extracted to /content


In [ ]:
# This code is uesful only when we want to delete a directory (as it is not manually possible)

# Specify the path to the folder you want to delete.
folder_path = '/content/semantic_masks'  # Change this to the path of the folder you need to remove.

# Delete the folder and everything inside it.
shutil.rmtree(folder_path)

## Main

In [4]:
# This part defines the paths for input images, the JSON annotation file, and the output directory
# for storing generated semantic masks. It also ensures the output directory exists before proceeding.

# Provide your updated paths
image_dir = '/content/images'  # Directory with input images
json_path = '/content/result.json'  # Path to your JSON file
semantic_mask_dir = '/content/semantic_masks/'  # Directory to save semantic segmentation masks

# Ensure output directory exists
os.makedirs(semantic_mask_dir, exist_ok=True)

In [5]:
# In this section, a dictionary is created to map class names to specific grayscale values. These values will
# be used to generate the semantic segmentation masks, with each class represented by a unique grayscale value.

# Grayscale values for semantic mask (ensuring contrast)
grayscale_mapping = {
    'Unlabelled': 0,       # Background
    'Boundary': 1,         # Edges or boundaries
    'Land': 2,             # Land areas
    'Poles': 3,           # Poles / Steel Modules
    'Solar_Panels': 4      # Solar panels
}

In [6]:
# This part loads the annotation data from the COCO-format JSON file and extracts mappings from category IDs to class names.
# It also organizes the annotations by image ID for easier access during mask generation.

# Load the JSON annotation file
with open(json_path) as f:
    annotations = json.load(f)

# Create a dictionary to map category ids to their names
category_mapping = {cat['id']: cat['name'] for cat in annotations['categories']}

# Create a dictionary to map image ids to their annotations
image_annotations = {}
for annotation in annotations['annotations']:
    image_id = annotation['image_id']
    if image_id not in image_annotations:
        image_annotations[image_id] = []
    image_annotations[image_id].append(annotation)

In [7]:
# This section creates a mapping of image IDs to their respective filenames,
# stripping any directory prefixes to ensure correct file handling during the mask generation process.

# Create a dictionary to map image ids to their filenames and strip directory prefixes
image_mapping = {img['id']: os.path.basename(img['file_name'].replace("images\\", "")) for img in annotations['images']}

In [ ]:
# Here, the script loops through each image, loads it, and initializes an empty semantic mask. If the image has annotations,
# it processes each polygon (representing a segmented object) and fills in the corresponding areas of the mask using grayscale values
# from the grayscale_mapping. The masks and images are then saved with sequentially named filenames.

# Loop through all images in the directory and create masks with corresponding names
for idx, (image_id, image_file) in enumerate(image_mapping.items(), start=1):
    # Construct image path
    image_path = os.path.join(image_dir, image_file)

    # Ensure the image file exists
    if not os.path.exists(image_path):
        print(f"Image file {image_file} not found in {image_dir}. Skipping.")
        continue

    # Load the input image
    input_image = cv2.imread(image_path)
    if input_image is None:
        print(f"Error loading image {image_file}. Skipping.")
        continue
    height, width, _ = input_image.shape

    # Initialize semantic mask for this image
    semantic_mask = np.zeros((height, width), dtype=np.uint8)  # Semantic Segmentation mask

    # Get annotations for this image
    if image_id in image_annotations:
        for annotation in image_annotations[image_id]:
            category_id = annotation['category_id']

            # Get the class name using category_id
            category_name = category_mapping.get(category_id, "Unknown")
            if category_name == "Unknown":
                print(f"Warning: Unknown category_id {category_id} found in annotations.")
            class_gray = grayscale_mapping.get(category_name, 0)  # Default to 0 (black) if class not found

            # Convert polygons into a format suitable for drawing
            for polygon in annotation['segmentation']:
                points = np.array(polygon, dtype=np.int32).reshape((-1, 1, 2))

                # Debug print
                print(f"Drawing polygon for category {category_name} with points: {points}")

                # Draw the polygon on the Semantic Segmentation mask with grayscale values
                cv2.fillPoly(semantic_mask, [points], class_gray)
    else:
        print(f"No annotations found for image_id: {image_id}")

    # Save the semantic mask with a sequential name
    new_image_name = f"image_no{idx}.JPG"
    new_mask_name = f"image_no{idx}.png"
    new_image_path = os.path.join(image_dir, new_image_name)
    semantic_mask_path = os.path.join(semantic_mask_dir, new_mask_name)

    # Rename the image file to the sequential name
    os.rename(image_path, new_image_path)

    # Save the mask
    cv2.imwrite(semantic_mask_path, semantic_mask)

print("Semantic masks created and saved successfully.")

# Download the 'images' and 'semantic_masks' directories into personal computer for further use. 